In [ ]:
import numpy as np
from collections import deque

## Part 1

In [ ]:
class robot_state():
    def __init__(self, pos: tuple, cost: int, dir: str):
        self.pos = pos
        self.cost = cost # cost after getting to that pos and in that dir
        self.dir = dir
        
    def update_pos(self,step:tuple):
        self.pos = tuple(np.array(self.pos)+ np.array(step))
        self.cost +=1 
        
    def update_dir(self, dir:str):
        self.dir = dir
        self.cost +=1000
        
    def clone(self):
        # Return a new instance with the same values
        return robot_state(pos=self.pos, cost=self.cost, dir=self.dir)
    
    def __repr__(self):
        return f"|pos = {np.copy(self.pos)}, dir = {self.dir}, cost = {self.cost}|"

In [ ]:
dirs_tup =  {
    "N": (-1, 0),
    "S": (1, 0),
    "W": (0,-1),
    "E": (0, 1)
    }

front_plus_neighbs = {
    "N": [dirs_tup["N"], dirs_tup["W"] ,dirs_tup["E"]],
    "S": [dirs_tup["S"], dirs_tup["W"] ,dirs_tup["E"]],
    "W": [dirs_tup["S"], dirs_tup["N"] ,dirs_tup["W"]],
    "E": [dirs_tup["S"], dirs_tup["E"] ,dirs_tup["N"]]
}


def is_dead_end(Map, robot):
    oth_dirs = front_plus_neighbs[robot.dir]
    rob_pos = robot.pos
    for dir in oth_dirs:
        new_pos =(rob_pos[0] + dir[0], rob_pos[1]+dir[1])
        if Map[new_pos]!="#":
            return False
    return True


In [ ]:
steps = {
    "N": {"F": ("update_pos", (-1, 0)), "L": ("update_dir", "W"), "R": ("update_dir", "E")},
    "S": {"F": ("update_pos", (1, 0)), "L": ("update_dir", "E"), "R": ("update_dir", "W")},
    "W": {"F": ("update_pos", (0, -1)), "L": ("update_dir", "S"), "R": ("update_dir", "N")},
    "E": {"F": ("update_pos", (0, 1)), "L": ("update_dir", "N"), "R": ("update_dir", "S")}
}

def find_best_path(Map: np.array, S_pos: tuple, E_pos: tuple, verbose: bool = False):
    robot_S = robot_state( S_pos, 0, "E")
    
    completed_paths = []
    Paths = deque( )
    Paths.append([robot_S])
    full_path_costs = []
    
    
    while len(Paths) >0:
        Paths = deque(sorted(Paths, key=lambda path: path[-1].cost))
        if verbose:
            print(f"\n--------------------------\nPaths = ")
            for path in Paths:
                print(path)
        partial_path = Paths.popleft()
        partial_path_pos_dir =[(rob.pos, rob.dir) for rob in partial_path]
        last_robot_state = partial_path[-1]
        last_dir = last_robot_state.dir
        actions = steps[last_dir]
        verbose and print(f"partial_path = {partial_path}")
        for action in actions: #F, L, R
            verbose and print(f"\nFor {last_robot_state} checking move {action}= {steps[last_dir][action]}")
            method_name, argument = steps[last_dir][action]
            #make clone
            new_robot_state = last_robot_state.clone()
            # Dynamically call the appropriate method on new state
            getattr(new_robot_state, method_name)(argument)
            verbose and print(f"new_robot updated: {new_robot_state}")
            
            if new_robot_state.cost > 102504:
                continue
                
            verbose and action !="F" and print(f"Facing towards{tuple(new_robot_state.pos + np.array(dirs_tup[actions[action][1]] ) )}" )
            
            
            # let's ignore the moves if we turned towards a wall
            if action != "F" and Map[tuple(new_robot_state.pos + np.array(dirs_tup[actions[action][1]] ) ) ] =="#":
                verbose and print("Turned towards wall, move discarded")
                continue
            
            # if we are arrived
            if tuple(new_robot_state.pos) ==E_pos:
                full_path_costs.append(new_robot_state.cost)
                verbose and print(f"Found path: {[[ [int(rob.pos[0])+1,int(rob.pos[1])+1] , rob.dir] for rob in partial_path]} with cost {new_robot_state.cost}")
                completed_paths.append(partial_path+[new_robot_state])
                not verbose and print(new_robot_state.cost)
                if len(full_path_costs)>5:
                    return full_path_costs, Map
                continue
            # if we have already been in same pos-dir, or we are trying to walk on a wall
            if (new_robot_state.pos, new_robot_state.dir) in partial_path_pos_dir or Map[tuple(new_robot_state.pos)]=="#":
                continue
            
            new_Paths=[]
            
            is_redundant = False
            for other_paths in list(Paths) + completed_paths + new_Paths:
                # Check if the new_robot_state is redundant
                is_redundant = False
                for other_rob_state in other_paths:
                    if (
                        other_rob_state.pos == new_robot_state.pos and 
                        other_rob_state.dir == new_robot_state.dir and 
                        other_rob_state.cost < new_robot_state.cost
                    ):
                        verbose and print("----Path removed because more expensive than an already computed one")
                        is_redundant = True
                        break  # Stop checking other_paths; new_robot_state is redundant
                
                if is_redundant:
                    break  # No need to check further other_paths, skip to the next iteration of the outer loop

            # If no redundant path exists, process the new_robot_state
            if not is_redundant:
                # Check for dead-end
                if is_dead_end(Map, new_robot_state):
                    Map[new_robot_state.pos] = "#"
                    verbose and print(f"Turning dead end in position {new_robot_state.pos} into #")
                else:
                    # Add the new state to new_Paths
                    verbose and print(f"To partial_path {partial_path} adding new state {new_robot_state}")
                    new_Paths.append(partial_path + [new_robot_state])

             
            new_Paths = [list(t) for t in set(tuple(inner_list) for inner_list in new_Paths)]             
            Paths.extend(new_Paths)
            
            # the above for loop will not append anything if Paths is empty
            if len(new_Paths) ==0 and len(Paths) ==0 :
                Paths.append(partial_path+[new_robot_state])
        
        for path in completed_paths:
            for state in path:
                Map[tuple(state.pos)] = "O"
        
    return full_path_costs, Map

In [ ]:
Map = []
with open('Day16_input_cleaned.txt') as f:
     for row,line in enumerate(f):
        Map.append([char for char in line.replace("\n", "")])
        if "S" in line:
            S_pos = (row, line.index("S"))
        if "E" in line:
            E_pos = (row, line.index("E"))
        
Map = np.array(Map)
costs, Map = find_best_path(Map, S_pos, E_pos, verbose =False)
print(Map)
print(min(costs), costs)


In [ ]:
n_circles = 0
for line in Map:
    n_circles +=sum(line =="O")
    
print(n_circles)
# 511 too low
# 723 too high

## Part 2